Build a regression model.

In [1]:
import pandas as pd
import math

In [2]:
station_df = pd.read_csv('station_info.tsv', sep='\t')
foursqur_df = pd.read_csv('foursqur_info.tsv', sep='\t')
yelp_df = pd.read_csv('yelp_info.tsv', sep='\t')

In [4]:
merged_data = foursqur_df.merge(yelp_df, on=['latitude','longitude','address'])
merged_data.head()

,Unnamed: 0_x,id_x,latitude,longitude,address,categories_x,Unnamed: 0_y,id_y,name,is_closed,rating,categories_y


Provide model output and an interpretation of the results. 

In [ ]:
import sqlite3

from sqlite3 import Error

def create_connection(path):
    connection = None
    try:
        connection = sqlite3.connect(path)
        print("Connection to SQLite DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

In [ ]:
connection = create_connection("sm_app.sqlite")


# Stretch

How can you turn the regression model into a classification model?